# Traitement des données

Pour réaliser les visualisations, nous sommes partis du fichier `base_martyr_clear.ods`. À partir de ce dernier, nous avons sélectionné manuellement les colonnes pertinentes afin de constituer un nouveau fichier `.ods` plus adapté à notre usage.

Les données ont ensuite été traitées pour en faciliter la lecture dans les visualisations. Cela a impliqué l’harmonisation de certaines valeurs et la gestion des incertitudes.

## Diagramme circulaire (Sunburst)

Avant le traitement automatisé en Python, deux colonnes supplémentaires — `aorum:gildingGenericTerm` et `aorum:gildingSpecificTerm` — ont été extraites du fichier `gilding_two_levels.ods` et intégrées manuellement à la feuille `dorure_clear` du fichier `base_martyr_clear.ods`. 
Ces colonnes permettent une structuration hiérarchique des motifs de dorure : un premier niveau générique, suivi d’un second niveau plus spécifique. Cette hiérarchisation est indispensable à la génération du diagramme circulaire de type *Sunburst*.

### Pipeline de traitement :

1. **Jointure** des feuilles `cartel_clear` et `dorure_clear` issues de `base_martyr_clear.ods` via la clé commune `dcterms:identifier`. Seules les colonnes suivantes sont conservées :
   - `dcterms:identifier`
   - `dcterms:spatial`
   - `aorum:gildingTechnique`
   - `aorum:gildingGenericTerm`
   - `aorum:gildingSpecificTerm`

2. **Gestion des incertitudes** :
   - Attribution d’un indicateur selon la présence d’un marqueur d'incertitude dans les champs d’origine :
     - `0` : aucune incertitude détectée
     - `1` : incertitude présente (ex. : `"Péninsule italienne (?)"` ou `"?"`)
   - Nettoyage des valeurs ambiguës ou multiples.

3. **Normalisation des techniques** :
   - Si plusieurs techniques sont indiquées (ex. : `"Mixtion ; Coquille"`), les lignes sont **dupliquées** pour dissocier chaque technique dans une ligne distincte.
   - Ce traitement permet de simplifier l'agrégation et de réduire la complexité des filtres dans les visualisations finales.




In [1]:
# Jointure des feuilles cartel_clear et dorure_clear issues de base_martyr_clear.ods

import pandas as pd
from odf.opendocument import OpenDocumentSpreadsheet
from odf.table import Table, TableRow, TableCell
from odf.text import P

ods_path = "base_martyr_clear.ods"

# Lire les feuilles
cartel_df = pd.read_excel(ods_path, sheet_name="cartel_clear", engine="odf")
dorures_df = pd.read_excel(ods_path, sheet_name="dorure_clear", engine="odf")

# Nettoyage des colonnes
cartel_df.columns = cartel_df.columns.str.strip()
dorures_df.columns = dorures_df.columns.str.strip()

# Colonnes à garder
cartel_cols = ['dcterms:identifier', 'dcterms:spatial']
dorures_cols = ['dcterms:identifier', 'aorum:gildingTechnique', 'aorum:gildingGenericTerm', 'aorum:gildingSpecificTerm']

# Vérification existence des colonnes
available_dorures_cols = [col for col in dorures_cols if col in dorures_df.columns]
available_cartel_cols = [col for col in cartel_cols if col in cartel_df.columns]

# Filtrage
cartel_filtered = cartel_df[available_cartel_cols]
dorures_filtered = dorures_df[available_dorures_cols]

# Fusion
merged_df = pd.merge(dorures_filtered, cartel_filtered, on='dcterms:identifier', how='left')

# Créer un nouveau document ODS
new_doc = OpenDocumentSpreadsheet()

# Convertir DataFrame en feuille ODS
table = Table(name="oeuvres_dorures_merge")
# En-têtes
header_row = TableRow()
for col in merged_df.columns:
    header_cell = TableCell()
    header_cell.addElement(P(text=col))
    header_row.addElement(header_cell)
table.addElement(header_row)
# Lignes
for _, row in merged_df.iterrows():
    tr = TableRow()
    for val in row:
        tc = TableCell()
        tc.addElement(P(text=str(val) if pd.notnull(val) else ""))
        tr.addElement(tc)
    table.addElement(tr)

# Ajouter la feuille au document
new_doc.spreadsheet.addElement(table)

# Sauvegarde
new_path = "base_sunburst_traitement_1.ods"
new_doc.save(new_path)

print(f"✅ Fichier modifié sauvegardé : {new_path}")

✅ Fichier modifié sauvegardé : base_sunburst_traitement_1.ods


In [3]:
# Script pour harmoniser les valeurs et détecter les incertitudes dans les colonnes
# relatives aux techniques de dorure et aux provenances géographiques.

import pandas as pd

# Paramètres
fichier_entree = "base_sunburst_traitement_1.ods"  # Le fichier ODS à traiter
nom_feuille = "oeuvres_dorures_merge"  # Nom de la feuille contenant les données
fichier_sortie = "base_sunburst_traitement_2.ods"  # Nom du fichier ODS en sortie

# Fonction pour harmoniser et détecter les incertitudes
def harmonise_et_incertitude(val):
    if pd.isna(val):
        return "", 0
    val_str = str(val)
    incertitude = int("?" in val_str or "(?)" in val_str)
    harmonise = val_str.replace("?", "").replace("()","").replace("(?)", "").strip()
    return harmonise, incertitude

# Traitement des données
# Lire le fichier ODS
df = pd.read_excel(fichier_entree, engine="odf", sheet_name=nom_feuille, header=0)

# Créer les colonnes pour la technique
df["aorum:harmonise"], df["aorum:incertitude"] = zip(*df["aorum:gildingTechnique"].map(harmonise_et_incertitude))

# Créer les colonnes pour la provenance
df["aorum:spatialharmonise"], df["aorum:spatialincertitude"] = zip(*df["dcterms:spatial"].map(harmonise_et_incertitude))

# Export
df.to_excel(fichier_sortie, engine="odf", index=False)

print(f"✅ Fichier modifié sauvegardé : {fichier_sortie}")

✅ Fichier modifié sauvegardé : base_sunburst_traitement_2.ods


In [4]:
# Fonction pour dupliquer les lignes lorsqu'une entrée contient plusieurs techniques de dorure
# Ceci permet de simplifier les filtres dans les visualisations en ayant une technique par ligne.

import pandas as pd

def nettoyer_techniques_or(fichier_entree, fichier_sortie, colonne="aorum:harmonise"):
    # Lire le fichier .ods avec pandas en utilisant le moteur 'odf'
    df = pd.read_excel(fichier_entree, engine="odf")

    # Convertir les valeurs de la colonne ciblée en chaînes de caractères et supprimer les espaces en début/fin
    df[colonne] = df[colonne].astype(str).str.strip()

    # Séparer les techniques multiples dans la colonne, séparées par un point-virgule (';')
    # La méthode str.split génère des listes, puis explode crée une ligne par élément de ces listes
    df = df.assign(**{
        colonne: df[colonne].str.split(';')
    }).explode(colonne)

    # Nettoyer à nouveau les espaces en début/fin sur la colonne après séparation
    df[colonne] = df[colonne].str.strip()

    # Réinitialiser l'index du DataFrame, car explode crée un index fragmenté
    df.reset_index(drop=True, inplace=True)

    # Exporter le DataFrame modifié dans un nouveau fichier .ods
    df.to_excel(fichier_sortie, engine="odf", index=False)
    print(f"✅ Fichier modifié sauvegardé : {fichier_sortie}")

# Définition des chemins d'entrée et de sortie
fichier_entree = "base_sunburst_traitement_2.ods"
fichier_sortie = "base_sunburst_traitement_3.ods"

# Exécution de la fonction avec les fichiers spécifiés
nettoyer_techniques_or(fichier_entree, fichier_sortie)

✅ Fichier modifié sauvegardé : base_sunburst_traitement_3.ods


## Diagramme de provenance géographique

La première étape consiste à extraire les colonnes pertinentes depuis le fichier `base_martyr_clear.ods`, à savoir :
- `dcterms:identifier`
- `dcterms:spatial`
- `aorum:cityOfCreation`

### Pipeline de traitement :

1. **Harmonisation des toponymes** :
   - Création de deux nouvelles colonnes : `cityOfCreation:harmonise` et `spatial:harmonise`.
   - Suppression des marqueurs d'incertitude (`?`, `(?)`) afin d'obtenir des noms de lieux standardisés pour les visualisations.

2. **Gestion des incertitudes** :
   - Génération de deux colonnes booléennes : `cityOfCreation:incertain` et `spatial:incertain`.
   - Attribution d’un indicateur selon la présence d’un marqueur d'incertitude dans les champs d’origine :
     - `0` : aucune incertitude détectée
     - `1` : incertitude présente (ex. : `"Rome (?)"` ou `"?"`)

Ce traitement permet d’exploiter de manière cohérente les données géographiques tout en conservant l’information sur leur fiabilité.

Un traitement manuel complémentaire a été nécessaire pour renseigner la colonne cityOfCreation:harmonise, en raison de l’absence de données pour certaines entrées. Les valeurs manquantes ont été imputées avec la mention standardisée "Ville inconnue (Péninsule italienne)", à titre d'exemple.


In [8]:
# Script pour harmoniser les noms des villes et aires géographiques
# et détecter les incertitudes marquées par "(?)" dans les données de provenance.

import ezodf
import pandas as pd
import re

# Paramètres
input_file = "base_diagramme_provenance.ods" # Fichier source ODS à charger
output_file = "base_diagramme_provenance_traitement_1.ods" # Fichier de sortie avec les colonnes ajoutées

# Chargement du fichier .ods
doc = ezodf.opendoc(input_file)
sheet = doc.sheets[0]  # On travaille sur la première feuille par défaut

# Extraction des données dans une liste de listes
data = [[cell.value for cell in row] for row in sheet.rows()]

# Conversion en DataFrame pandas
df = pd.DataFrame(data)

# Utiliser la première ligne comme noms de colonnes
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

# Vérification que les colonnes nécessaires existent
required_columns = ["dcterms:spatial", "aorum:cityOfCreation"]
missing_cols = [col for col in required_columns if col not in df.columns]
if missing_cols:
    raise KeyError(f"Colonnes manquantes dans le fichier : {missing_cols}")

# Fonction de nettoyage et détection des incertitudes
def harmonise(val):
    """
    Supprime la mention d'incertitude '(?)' et les espaces superflus autour.
    Retourne la valeur telle quelle si ce n'est pas une chaîne.
    """
    if not isinstance(val, str):
        return val
    return re.sub(r"\s*\(\?\)", "", val).strip()

def incertain(val):
    """
    Renvoie 1 si la valeur contient '(?)', sinon 0.
    Retourne 0 si ce n'est pas une chaîne.
    """
    if not isinstance(val, str):
        return 0
    return 1 if "(?)" in val else 0

# Traitement des données
df["spatial:harmonise"] = df["dcterms:spatial"].apply(harmonise)
df["spatial:incertain"] = df["dcterms:spatial"].apply(incertain)

df["cityOfCreation:harmonise"] = df["aorum:cityOfCreation"].apply(harmonise)
df["cityOfCreation:incertain"] = df["aorum:cityOfCreation"].apply(incertain)

# Sauvegarde du fichier modifié
df.to_excel(output_file, engine="odf", index=False)

print(f"✅ Fichier modifié enregistré : {output_file}")

✅ Fichier modifié enregistré : base_diagramme_provenance_traitement_1.ods


## Histogramme de la production artistique

Les données utilisées pour cette visualisation ont été extraites depuis le fichier `base_martyr_clear.ods`, en ne conservant que les colonnes suivantes :
- `dcterms:identifier`
- `dcterms:spatial`
- `aorum:dateHidden`

### Pipeline de traitement :

1. **Harmonisation des aires géographiques** :
   - Création d’une colonne `spatial:harmonise` à partir de `dcterms:spatial`.
   - Suppression des marqueurs d’incertitude (`?`, `(?)`) afin de ne conserver que les noms de régions ou zones géographiques normalisés.

2. **Détection des incertitudes** :
   - Ajout d’une colonne booléenne `spatial:incertitude`.
   - Codage binaire selon la présence d’un indicateur d’incertitude :
     - `0` : aucune incertitude (valeur propre)
     - `1` : incertitude présente (ex. : `"Péninsule italienne (?)"`)

Ce prétraitement garantit la cohérence des données spatiales tout en préservant l'information liée à leur fiabilité, essentielle pour une lecture critique de l’histogramme.


In [63]:
# Harmonisation et gestion de l'incertitude : ajout de deux colonnes pour harmoniser les aires géographiques et gérer les incertitudes

import pandas as pd
import re

# Charger le fichier .ods
input_file = "base_histogramme_production.ods"
df = pd.read_excel(input_file, engine="odf")

# Fonctions utilitaires
def nettoyer_spatial(val):
    if pd.isna(val):
        return val
    # Supprime les "?" et "(?)", tout en conservant le reste
    return re.sub(r"\s*\(\?\)|\?", "", val).strip()

def contient_incertitude(val):
    if pd.isna(val):
        return 0
    return 1 if "?" in val else 0

# Ajouter les deux nouvelles colonnes
df["aorum:spatialharmonise"] = df["dcterms:spatial"].apply(nettoyer_spatial)
df["aorum:spatialincertitude"] = df["dcterms:spatial"].apply(contient_incertitude)

# Sauvegarder dans un nouveau fichier
output_file = "base_histogramme_production_traitement_1.ods"
df.to_excel(output_file, engine="odf", index=False)

print(f"Fichier modifié enregistré sous : {output_file}")


Fichier modifié enregistré sous : base_histogramme_production_traitement_1.ods
